In [1]:
from dateutil.parser import parse 
import numpy as np
import pandas as pd
import os
import time
from datetime import datetime
from datetime import date
import unicodedata

In [2]:
fileLocation = "C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\Datasets\\CleaningStage\\stage1PreProcessedDF_Apr-12-2020.csv"
folder_path = "C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\Datasets\\CleaningStage"

# Functions

## Helper Functions to save dataframe to path

In [3]:
# Helper method for save_to_file

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [4]:
# Function: Save Dataframe to CSV

def save_to_file(dataFrame, folderPath):
    today = date.today()
    todayDate = today.strftime("%b-%d-%Y")
    dataFrame.to_csv(folderPath+ '\\' + get_df_name(dataFrame) + '_' + todayDate  + '.csv',index=False, encoding="utf-8-sig" )

## Function to strip the empty spaces

In [5]:
# Function to return a stripped list. To be applied on column dataframes.
def strip_spaces_from_list(row_list):
    stripped_list = [element.strip() for element in row_list]
    return stripped_list

## Function to perform value_count on a list column

In [6]:
# Function to returns a dataframe with sorted descending frequency count of occurences in a list column dataframe

def value_counts_from_list_column(df,columnName):
    explode = pd.Series([element 
               for sublist in df[columnName] 
               for element in sublist])

    explode_value_count_df = explode.value_counts().sort_index().rename_axis(columnName).reset_index(name='frequency').sort_values(by='frequency',ascending = False)
    return explode_value_count_df

## Function perform one hot encoding on a multiclass label containing the values in lists

In [7]:
# One Hot encoding of all category value based classes

from sklearn.preprocessing import MultiLabelBinarizer

def one_hot_encode_and_merge_orginalDF(df,columnName,renameCol):
    
    mlb = MultiLabelBinarizer()
    
    # MultiLableBinarizer to transform the chosen column into multiple columns
    one_hot_df = pd.DataFrame(mlb.fit_transform(df[columnName]),columns= renameCol + mlb.classes_, index=df.index)
    
#     # Rename columns
#     category_classes = renameCol + mlb.classes_
#     Merge columns and remove the transformed the column
    
#     df = df.assign(**{cls: one_hot_df[cls].to_numpy() for cls in category_classes})

    # Merge columns and remove the transformed the column
    df = pd.concat([toBeExplodeddf,one_hot_df],axis=1)
    df.drop(columns= columnName,inplace= True)
    return df

## Function to find the optimal value count length given the percentage required

In [8]:
def find_optimal_value_count(data,percent):
    accuracy = round((percent/100),2)
    count = 1
    sel_accuracy = 0
    while(accuracy > sel_accuracy):
        sel_accuracy = data[:count].sum() / data.sum()
        count = count + 1
    return count-1

## Function to provide a list of values which are not in the optimal value count length to replace them with 'Other'

In [9]:
def other_value_list(df,valueColumn,optimal_value):
    all_list = df[valueColumn].values.tolist()
    opt_list = df[valueColumn][:optimal_value].values.tolist()
    other_values_list = [element 
                        for element in all_list 
                        if element not in opt_list]
    return other_values_list

# Data Cleaning and Preprocessing

In [10]:
# Import as Dataframe

toBeExplodeddf = pd.read_csv(fileLocation,encoding="utf8")
toBeExplodeddf.columns = map(str.lower, toBeExplodeddf.columns)
print("\nInitial Dataframe Shape : ",str(toBeExplodeddf.shape))
#toBeExplodeddf = toBeExplodeddf.drop(columns=['Directed_by'])


Initial Dataframe Shape :  (621943, 24)


In [11]:
toBeExplodeddf.columns

Index(['movie_id', 'title', 'review_count_user', 'review_count_critic',
       'metascore', 'rating_value', 'rating_count', 'date_published',
       'duration', 'title_year', 'genre_tags', 'director_names',
       'director_ids', 'country', 'language', 'release_date', 'budget',
       'opening_weekend', 'gross', 'runtime', 'color', 'directed_by',
       'release_year', 'movie_year'],
      dtype='object')

In [12]:
save = toBeExplodeddf.copy()

## 1. Opening Weekend : Data cleaning & Preprocessing

### 1a) Extracting denomination and amount from the Opening Weekend

In [13]:
toBeExplodeddf['opening_weekend'].value_counts()

None                                                             611484
         $5,954              (USA)      (10 August 2001)              2
         $4,104,298              (USA)      (24 March 2000)           2
         $1,500              (USA)      (4 November 2016)             2
         $157,049              (USA)      (16 May 1997)               2
                                                                  ...  
         $73,394              (USA)      (8 May 2009)                 1
         $44,313              (USA)      (28 May 2010)                1
         $15,810              (USA)      (14 September 2007)          1
         $2,270,290              (USA)      (25 October 2002)         1
         $3,653,281              (USA)      (29 October 1999)         1
Name: opening_weekend, Length: 10454, dtype: int64

#### The opening weekend contains comma values. Remove them from the series

In [14]:
# The opening weekend contains comma values. Remove them from the series

toBeExplodeddf['opening_weekend'] = toBeExplodeddf['opening_weekend'].str.replace(',','').str.lstrip()
toBeExplodeddf['opening_weekend'].value_counts(dropna = False).head()

None                                                611484
$4104298              (USA)      (24 March 2000)         2
$5954              (USA)      (10 August 2001)           2
$5910              (USA)      (5 March 2010)             2
$1500              (USA)      (4 November 2016)          2
Name: opening_weekend, dtype: int64

#### Remove the text within the brackets

In [15]:
# Remove the text within the brackets (estimated)

toBeExplodeddf['opening_weekend'] = toBeExplodeddf['opening_weekend'].str.extract('(.*?)\(.*?\)')
toBeExplodeddf['opening_weekend'] = toBeExplodeddf['opening_weekend'].str.strip()

# # Replace Nan values with None
# toBeExplodeddf['opening_weekend'] = toBeExplodeddf['opening_weekend'].fillna('None')

toBeExplodeddf['opening_weekend'].value_counts(dropna = False).head()

NaN      611484
$6000         5
$5000         5
$7100         4
$1500         4
Name: opening_weekend, dtype: int64

#### Replace currency symbols with 3 letter currency equivalent

In [16]:
# Replace currency symbols with 3 letter currency equivalent

replace_currency = { '\$' : 'USD', '\£' : 'GBP', unicodedata.lookup('EURO SIGN') : 'EUR' }
toBeExplodeddf.replace({'opening_weekend': replace_currency},inplace=True,regex=True)
toBeExplodeddf['opening_weekend'].value_counts(dropna = False).head()

NaN        611484
USD5000         5
USD6000         5
USD1500         4
USD488          4
Name: opening_weekend, dtype: int64

#### Extract opening weekend value amount and denomination into two new columns

In [17]:
# Extract opening weekend value amount and denomination into two new columns

toBeExplodeddf['opening_weekend_value'] = toBeExplodeddf['opening_weekend'].str.extract('(\d+)')
toBeExplodeddf['opening_weekend_currency_denomination'] = toBeExplodeddf['opening_weekend'].str.extract('(^\D+)')
toBeExplodeddf['opening_weekend_currency_denomination'] = toBeExplodeddf['opening_weekend_currency_denomination'].str.strip()
toBeExplodeddf['opening_weekend_currency_denomination'].value_counts(dropna = False)

NaN    611484
USD      9402
GBP       962
DEM        48
ITL        28
JPY         5
CAD         5
SEK         4
SGD         4
AUD         1
Name: opening_weekend_currency_denomination, dtype: int64

### 1b) Pre processing - One hot encoding

In [18]:
toBeExplodeddf.shape

(621943, 26)

In [19]:
# one hot encoding of opening_weekend_currency_denomination
toBeExplodeddf = pd.concat([toBeExplodeddf, pd.get_dummies(toBeExplodeddf['opening_weekend_currency_denomination'],
                                                           prefix='open_wk_cur')],axis=1)
toBeExplodeddf.drop(['opening_weekend','opening_weekend_currency_denomination'],axis=1, inplace=True)
toBeExplodeddf.head(2)

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,opening_weekend_value,open_wk_cur_AUD,open_wk_cur_CAD,open_wk_cur_DEM,open_wk_cur_GBP,open_wk_cur_ITL,open_wk_cur_JPY,open_wk_cur_SEK,open_wk_cur_SGD,open_wk_cur_USD
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,NaN,0,0,0,0,0,0,0,0,0
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,None,None,6.5,148,1892-10-28,5min,1892,...,NaN,0,0,0,0,0,0,0,0,0


In [20]:
toBeExplodeddf.shape

(621943, 33)

In [21]:
toBeExplodeddf.columns

Index(['movie_id', 'title', 'review_count_user', 'review_count_critic',
       'metascore', 'rating_value', 'rating_count', 'date_published',
       'duration', 'title_year', 'genre_tags', 'director_names',
       'director_ids', 'country', 'language', 'release_date', 'budget',
       'gross', 'runtime', 'color', 'directed_by', 'release_year',
       'movie_year', 'opening_weekend_value', 'open_wk_cur_AUD',
       'open_wk_cur_CAD', 'open_wk_cur_DEM', 'open_wk_cur_GBP',
       'open_wk_cur_ITL', 'open_wk_cur_JPY', 'open_wk_cur_SEK',
       'open_wk_cur_SGD', 'open_wk_cur_USD'],
      dtype='object')

## 2) Gross : Data cleaning & Preprocessing

### 2a) Data Cleaning

#### Verifying the count of improper records - Before

In [22]:
# Count of records which match the improper pattern

print("\nRows containing Also Known as in Gross column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['gross'].str.contains(pat = 'Also Known As',na=False)]))

print("Rows containing See more in Gross column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['gross'].str.contains(pat = 'See more',na=False)]))

print("Rows containing Filming Location in Gross column before processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['gross'].str.contains(pat = 'Filming',na=False)]))


Rows containing Also Known as in Gross column before processing : 6
Rows containing See more in Gross column before processing : 520
Rows containing Filming Location in Gross column before processing: 99


#### Cleaning the improper records based on the pattern

In [23]:
# Pattern : Also Known as & See more. Clear the ones with texts from Color Column

toBeExplodeddf.loc[toBeExplodeddf['gross'].str.contains(pat = 'Also Known As',na=False),'gross'] = "None"
toBeExplodeddf.loc[toBeExplodeddf['gross'].str.contains(pat = 'See more',na=False),'gross'] = "None"
toBeExplodeddf.loc[toBeExplodeddf['gross'].str.contains(pat = 'Filming',na=False),'gross'] = "None"

#### Verifying the count of improper records - After cleaning

In [24]:
# Count of records which match the improper pattern

print("\nRows containing Also Known as in Gross column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['gross'].str.contains(pat = 'Also Known As',na=False)]))

print("Rows containing See more in Gross column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['gross'].str.contains(pat = 'See more',na=False)]))

print("Rows containing Filming Location in Gross column before processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['gross'].str.contains(pat = 'Filming',na=False)]))


Rows containing Also Known as in Gross column before processing : 0
Rows containing See more in Gross column before processing : 0
Rows containing Filming Location in Gross column before processing: 0


### 2b) Extracting denomination and amount from the Gross

#### The gross contains comma values. Remove them from the series

In [25]:
# The gross contains comma values. Remove them from the series

toBeExplodeddf['gross'] = toBeExplodeddf['gross'].str.replace(',','').str.lstrip()
toBeExplodeddf['gross'].value_counts(dropna = False).head()

None                           609294
$1200000              (USA)        14
$1000000              (USA)        14
$3000000              (USA)        14
$1500000              (USA)        11
Name: gross, dtype: int64

#### Remove the text within the brackets (estimated)

In [26]:
# Remove the text within the brackets (estimated)

toBeExplodeddf['gross'] = toBeExplodeddf['gross'].str.extract('(.*?)\(.*?\)')
toBeExplodeddf['gross'] = toBeExplodeddf['gross'].str.strip()
toBeExplodeddf['gross'].value_counts(dropna = False).head()

NaN         609294
$3000000        14
$1200000        14
$1000000        14
$1500000        11
Name: gross, dtype: int64

#### Replace currency symbols with 3 letter currency equivalent

In [27]:
# Replace currency symbols with 3 letter currency equivalent

replace_currency = { '\$' : 'USD', '\£' : 'GBP', unicodedata.lookup('EURO SIGN') : 'EUR' }
toBeExplodeddf.replace({'gross': replace_currency},inplace=True,regex=True)
toBeExplodeddf['gross'].value_counts(dropna = False).head()

NaN           609294
USD1000000        14
USD1200000        14
USD3000000        14
USD1500000        11
Name: gross, dtype: int64

#### Extract gross amount and denomination into two new columns

In [28]:
# Extract gross amount and denomination into two new columns

toBeExplodeddf['gross_value'] = toBeExplodeddf['gross'].str.extract('(\d+)')
toBeExplodeddf['gross_currency_denomination'] = toBeExplodeddf['gross'].str.extract('(^\D+)')
# toBeExplodeddf['gross_currency_denomination'] = toBeExplodeddf['gross_currency_denomination'].str.strip().astype(str)
toBeExplodeddf['gross_currency_denomination'] = toBeExplodeddf['gross_currency_denomination'].str.strip()

# Hard coding - To remove i
toBeExplodeddf.loc[toBeExplodeddf['gross_currency_denomination'].str.contains(pat = 'i',na=False),'gross_currency_denomination'] = np.nan
toBeExplodeddf['gross_currency_denomination'].value_counts(dropna = False)

NaN    609295
USD     12648
Name: gross_currency_denomination, dtype: int64

### 2c) Preprocessing the data - One hot encoding

In [29]:
# one hot encoding of gross_currency_denomination
toBeExplodeddf = pd.concat([toBeExplodeddf, pd.get_dummies(toBeExplodeddf['gross_currency_denomination'],
                                                           prefix='gross_cur')],axis=1)
toBeExplodeddf.drop(['gross','gross_currency_denomination'],axis=1, inplace=True)
toBeExplodeddf.head(2)

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,open_wk_cur_CAD,open_wk_cur_DEM,open_wk_cur_GBP,open_wk_cur_ITL,open_wk_cur_JPY,open_wk_cur_SEK,open_wk_cur_SGD,open_wk_cur_USD,gross_value,gross_cur_USD
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,0,0,0,0,0,0,0,0,NaN,0
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,None,None,6.5,148,1892-10-28,5min,1892,...,0,0,0,0,0,0,0,0,NaN,0


In [30]:
toBeExplodeddf.shape

(621943, 34)

In [31]:
toBeExplodeddf.columns

Index(['movie_id', 'title', 'review_count_user', 'review_count_critic',
       'metascore', 'rating_value', 'rating_count', 'date_published',
       'duration', 'title_year', 'genre_tags', 'director_names',
       'director_ids', 'country', 'language', 'release_date', 'budget',
       'runtime', 'color', 'directed_by', 'release_year', 'movie_year',
       'opening_weekend_value', 'open_wk_cur_AUD', 'open_wk_cur_CAD',
       'open_wk_cur_DEM', 'open_wk_cur_GBP', 'open_wk_cur_ITL',
       'open_wk_cur_JPY', 'open_wk_cur_SEK', 'open_wk_cur_SGD',
       'open_wk_cur_USD', 'gross_value', 'gross_cur_USD'],
      dtype='object')

In [32]:
# toBeExplodeddf['Gross'].value_counts().to_csv('tst.csv')

## 3) Budget : Data cleaning & Preprocessing

### 3a) Data Cleaning

#### Verifying the count of improper records - Before

In [33]:
# Count of records which match the improper pattern

print("\nRows containing Also Known as in budget column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['budget'].str.contains(pat = 'Also Known As',na=False)]))

print("Rows containing See more in budget column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['budget'].str.contains(pat = 'See more',na=False)]))


Rows containing Also Known as in budget column before processing : 2
Rows containing See more in budget column before processing : 78


#### Cleaning the improper records based on the pattern

In [34]:
# Pattern : Also Known as & See more. Clear the ones with texts from budget Column

toBeExplodeddf.loc[toBeExplodeddf['budget'].str.contains(pat = 'Also Known As',na=False),'budget'] = "None"
toBeExplodeddf.loc[toBeExplodeddf['budget'].str.contains(pat = 'See more',na=False),'budget'] = "None"

#### Verifying the count of improper records - After cleaning

In [35]:
# Count of records which match the improper pattern

print("\nRows containing Also Known as in budget column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['budget'].str.contains(pat = 'Also Known As',na=False)]))

print("Rows containing See more in budget column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['budget'].str.contains(pat = 'See more',na=False)]))


Rows containing Also Known as in budget column before processing : 0
Rows containing See more in budget column before processing : 0


### 3b) Extracting denomination and amount from the budget

In [36]:
# The budget contains comma values. Remove them from the series

toBeExplodeddf['budget'] = toBeExplodeddf['budget'].str.replace(',','').str.lstrip()
toBeExplodeddf['budget'].value_counts(dropna = False).head()

None                                 552348
$10000              (estimated)        2379
$1000000              (estimated)      1515
$100000              (estimated)       1363
$5000              (estimated)         1283
Name: budget, dtype: int64

In [37]:
print("Amount of rows without a budget value : ", toBeExplodeddf['budget'].str.contains('None').sum())

Amount of rows without a budget value :  552348


In [38]:
# Remove the text within the brackets (estimated)

toBeExplodeddf['budget'] = toBeExplodeddf['budget'].str.extract('(.*?)\(.*?\)')
toBeExplodeddf['budget'] = toBeExplodeddf['budget'].str.strip()
toBeExplodeddf['budget'].value_counts(dropna = False).head()

NaN         552351
$10000        2379
$1000000      1515
$100000       1363
$5000         1283
Name: budget, dtype: int64

In [39]:
# Replace currency symbols with 3 letter currency equivalent

replace_currency = { '\$' : 'USD', '\£' : 'GBP', unicodedata.lookup('EURO SIGN') : 'EUR' }
toBeExplodeddf.replace({'budget': replace_currency},inplace=True,regex=True)
toBeExplodeddf['budget'].value_counts(dropna = False).head()

NaN           552351
USD10000        2379
USD1000000      1515
USD100000       1363
USD5000         1283
Name: budget, dtype: int64

In [40]:
# Extract budget amount and denomination into two new columns

toBeExplodeddf['budget_value'] = toBeExplodeddf['budget'].str.extract('(\d+)')
toBeExplodeddf['budget_currency_denomination'] = toBeExplodeddf['budget'].str.extract('(^\D+)')
# toBeExplodeddf['budget_currency_denomination'] = toBeExplodeddf['budget_currency_denomination'].str.strip().astype(str)
toBeExplodeddf['budget_currency_denomination'] = toBeExplodeddf['budget_currency_denomination'].str.strip()
toBeExplodeddf['budget_currency_denomination'].value_counts(dropna = False)

NaN    552351
USD     42972
EUR      8500
GBP      4973
CAD      3307
        ...  
CYP         1
KES         1
BOB         1
UYU         1
LUF         1
Name: budget_currency_denomination, Length: 107, dtype: int64

In [41]:
print("\nDataframe Shape After extracing budget values : ",str(toBeExplodeddf.shape))


Dataframe Shape After extracing budget values :  (621943, 36)


### 3c) Preprocessing the data - Optimal value count & One hot encoding

In [42]:
toBeExplodeddf['budget_currency_denomination'].value_counts()

USD    42972
EUR     8500
GBP     4973
CAD     3307
INR     1916
       ...  
LUF        1
UYU        1
GTQ        1
BND        1
PEN        1
Name: budget_currency_denomination, Length: 106, dtype: int64

In [43]:
budget_country_value_count = toBeExplodeddf['budget_currency_denomination'].value_counts().rename_axis('budget_currency_denomination').reset_index(name='frequency')
print("Number of unique countries for budget : ", len(budget_country_value_count))

Number of unique countries for budget :  106


#### Optimal value count for Budget Country list

In [44]:
required_percent = 90
optimal_value_count_budget = find_optimal_value_count(budget_country_value_count['frequency'],required_percent)
print("The amount of languages required to reach " + str(required_percent) + 
      " percentage is " + str(optimal_value_count_budget) + " countries budget currency")

The amount of languages required to reach 90 percentage is 6 countries budget currency


#### A list of budget currency of a country which are not in the optimal value count length to replace them with 'Other'

In [45]:
other_budget_countries_list = other_value_list(budget_country_value_count ,'budget_currency_denomination',optimal_value_count_budget)
print("Countries to be replaced with Other : ", len(other_budget_countries_list))

Countries to be replaced with Other :  100


#### Function to replace budget currency with other

In [46]:
toBeExplodeddf['budget_currency_denomination'].value_counts(dropna = False)

NaN    552351
USD     42972
EUR      8500
GBP      4973
CAD      3307
        ...  
CYP         1
KES         1
BOB         1
UYU         1
LUF         1
Name: budget_currency_denomination, Length: 107, dtype: int64

#### Replace other budget currency denomination with 'Other'

In [47]:
# Replace other countries with 'Other'
toBeExplodeddf.loc[toBeExplodeddf['budget_currency_denomination'].
                   isin(other_budget_countries_list),
                   'budget_currency_denomination'] = np.nan

In [48]:
toBeExplodeddf['budget_currency_denomination'].value_counts(dropna = False)

NaN    558766
USD     42972
EUR      8500
GBP      4973
CAD      3307
INR      1916
AUD      1509
Name: budget_currency_denomination, dtype: int64

#### one hot encoding of budget_currency_denomination

In [49]:
# one hot encoding of gross_currency_denomination

print("\nDataframe Shape Before One hot Encoding budget : ",str(toBeExplodeddf.shape))

toBeExplodeddf = pd.concat([toBeExplodeddf, pd.get_dummies(toBeExplodeddf['budget_currency_denomination'],
                                                           prefix='budget_cur')],axis=1)
toBeExplodeddf.drop(['budget','budget_currency_denomination'],axis=1, inplace=True)

print("\nDataframe Shape After One hot Encoding budget : ",str(toBeExplodeddf.shape))


Dataframe Shape Before One hot Encoding budget :  (621943, 36)

Dataframe Shape After One hot Encoding budget :  (621943, 40)


In [50]:
toBeExplodeddf.columns

Index(['movie_id', 'title', 'review_count_user', 'review_count_critic',
       'metascore', 'rating_value', 'rating_count', 'date_published',
       'duration', 'title_year', 'genre_tags', 'director_names',
       'director_ids', 'country', 'language', 'release_date', 'runtime',
       'color', 'directed_by', 'release_year', 'movie_year',
       'opening_weekend_value', 'open_wk_cur_AUD', 'open_wk_cur_CAD',
       'open_wk_cur_DEM', 'open_wk_cur_GBP', 'open_wk_cur_ITL',
       'open_wk_cur_JPY', 'open_wk_cur_SEK', 'open_wk_cur_SGD',
       'open_wk_cur_USD', 'gross_value', 'gross_cur_USD', 'budget_value',
       'budget_cur_AUD', 'budget_cur_CAD', 'budget_cur_EUR', 'budget_cur_GBP',
       'budget_cur_INR', 'budget_cur_USD'],
      dtype='object')

In [51]:
# toBeExplodeddf['budget_currency_denomination'].to_csv('tst.csv', encoding="utf-8-sig")

## 4) Color : Data cleaning & Pre processing

### 4a) Data cleaning

#### Verifying the count of improper records - Before

In [52]:
# Count of records which match the improper pattern

print("\nRows containing Also Known as in color column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['color'].str.contains(pat = 'Also Known As',na=False)]))

print("Rows containing See more in color column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['color'].str.contains(pat = 'See more',na=False)]))

print("Rows containing improper records in color column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['color'].str.contains(pat = 'USA',na=False)]))


Rows containing Also Known as in color column before processing : 2
Rows containing See more in color column before processing : 226
Rows containing improper records in color column before processing : 34


#### Cleaning the improper records based on the pattern

In [53]:
# Pattern : Also Known as & See more. Clear the ones with texts from Color Column

toBeExplodeddf.loc[toBeExplodeddf['color'].str.contains(pat = 'Also Known As',na=False),'color'] = "None"
toBeExplodeddf.loc[toBeExplodeddf['color'].str.contains(pat = 'See more',na=False),'color'] = "None"
toBeExplodeddf.loc[toBeExplodeddf['color'].str.contains(pat = 'USA',na=False),'color'] = "None"
toBeExplodeddf.loc[toBeExplodeddf['color'].str.contains(pat = 'Black',na=False),'color'] = "Black and White"
toBeExplodeddf.loc[toBeExplodeddf['color'].str.contains(pat = 'Color',na=False),'color'] = "Color"

#### Verifying the count of improper records - After cleaning

In [54]:
# Count of records which match the improper pattern

print("\nRows containing Also Known as in color column after processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['color'].str.contains(pat = 'Also Known As',na=False)]))

print("Rows containing See more in color column after processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['color'].str.contains(pat = 'See more',na=False)]))

print("Rows containing other improper records in color column before processing :", 
      len(toBeExplodeddf.loc[toBeExplodeddf['color'].str.contains(pat = 'USA',na=False)]))


Rows containing Also Known as in color column after processing : 0
Rows containing See more in color column after processing: 0
Rows containing other improper records in color column before processing : 0


### 4b) Pre processing - One hot encoding

#### Split into lists

In [55]:
toBeExplodeddf['color'] = toBeExplodeddf['color'].str.split("|", expand = False)

#### Remove trailing spaces in Color

In [56]:
# Remove trailing spaces in Color

toBeExplodeddf['color'] = toBeExplodeddf['color'].apply(strip_spaces_from_list)
toBeExplodeddf['color'].head()

0    [Black and White]
1              [Color]
2              [Color]
3              [Color]
4    [Black and White]
Name: color, dtype: object

#### Explode the Color to perform a value_count. Helps us understand the amount of features we would be adding.

In [57]:
# Explode the Color to perform a value_count. Helps us understand the amount of features we would be adding.

color_value_count = value_counts_from_list_column(toBeExplodeddf,'color')
print("Number of unique Colors : ", len(color_value_count))

Number of unique Colors :  3


In [58]:
color_value_count.head(10)

,color,frequency
1,Color,439973
2,None,103117
0,Black and White,78853


#### One Hot encoding of all category value based classes

In [59]:
# One Hot encoding of all category value based classes

print("\nDataframe Shape Before One hot Encoding color : ",str(toBeExplodeddf.shape))
toBeExplodeddf = one_hot_encode_and_merge_orginalDF(toBeExplodeddf,columnName ='color',renameCol = 'color_')
print("\nDataframe Shape After One hot Encoding Color : ",str(toBeExplodeddf.shape))


Dataframe Shape Before One hot Encoding color :  (621943, 40)

Dataframe Shape After One hot Encoding Color :  (621943, 42)


In [60]:
toBeExplodeddf.columns

Index(['movie_id', 'title', 'review_count_user', 'review_count_critic',
       'metascore', 'rating_value', 'rating_count', 'date_published',
       'duration', 'title_year', 'genre_tags', 'director_names',
       'director_ids', 'country', 'language', 'release_date', 'runtime',
       'directed_by', 'release_year', 'movie_year', 'opening_weekend_value',
       'open_wk_cur_AUD', 'open_wk_cur_CAD', 'open_wk_cur_DEM',
       'open_wk_cur_GBP', 'open_wk_cur_ITL', 'open_wk_cur_JPY',
       'open_wk_cur_SEK', 'open_wk_cur_SGD', 'open_wk_cur_USD', 'gross_value',
       'gross_cur_USD', 'budget_value', 'budget_cur_AUD', 'budget_cur_CAD',
       'budget_cur_EUR', 'budget_cur_GBP', 'budget_cur_INR', 'budget_cur_USD',
       'color_Black and White', 'color_Color', 'color_None'],
      dtype='object')

## 5) Country - Data cleaning & Pre processing

### 5a) Data cleaning

#### Verifying the count of improper records - Before

In [61]:
# Count of records which match the improper pattern

print("Rows containing Also Known as in country column before processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'Also Known As',na=False)]))

print("Rows containing See more in country column before processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'See more',na=False)]))

print("Rows containing Filming Location in country column before processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'Filming',na=False)]))

Rows containing Also Known as in country column before processing: 145
Rows containing See more in country column before processing: 439
Rows containing Filming Location in country column before processing: 136


#### Cleaning the improper records based on the pattern

In [62]:
# Pattern : Also Known as & See more. Clear the ones with texts for Country Column

toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'Also Known As',na=False),'country'] = "Other"
toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'See more',na=False),'country'] = "Other"

# Pattern : Filming Locations
toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'USA',na=False),'country'] = "USA"
toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'UK',na=False),'country'] = "UK"
toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'Australia',na=False),'country'] = "Australia"
toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'Canada',na=False),'country'] = "Canada"
toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'Spain',na=False),'country'] = "Spain"
toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'Denmark',na=False),'country'] = "Denmark"

#### Verifying the count of improper records - After cleaning

In [63]:
# Count of records which match the improper pattern

print("\nRows containing Also Known as in country column after processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'Also Known As',na=False)]))

print("Rows containing See more in country column after processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'See more',na=False)]))

print("Rows containing Filming Location in country column after processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['country'].str.contains(pat = 'Filming',na=False)]))


Rows containing Also Known as in country column after processing: 0
Rows containing See more in country column after processing: 0
Rows containing Filming Location in country column after processing: 0


### 5b) Pre processing - Optimal Value & One hot encoding

#### Split into lists

In [64]:
toBeExplodeddf['country'] = toBeExplodeddf['country'].str.split("|", expand = False)

#### Remove trailing spaces in Country

In [65]:
# Remove trailing spaces in country
toBeExplodeddf['country'] = toBeExplodeddf['country'].apply(strip_spaces_from_list)
toBeExplodeddf['country'].head()

0       [USA]
1    [France]
2    [France]
3    [France]
4       [USA]
Name: country, dtype: object

#### Explode the country to perform a value_count. Helps us understand the amount of features we would be adding.

In [66]:
# Explode the country to perform a value_count. Helps us understand the amount of features we would be adding.

country_value_count = value_counts_from_list_column(toBeExplodeddf,'country')
print("Number of unique Countries : ", len(country_value_count))

Number of unique Countries :  220


#### Optimal value count for Country

In [67]:
required_percent = 90
optimal_value_count_country = find_optimal_value_count(country_value_count['frequency'],required_percent)
print("The amount of countries required to reach " + str(required_percent) + 
      " percentage is " + str(optimal_value_count_country) + " countries")

The amount of countries required to reach 90 percentage is 24 countries


#### A list of countries which are not in the optimal value count length to replace them with 'Other'

In [68]:
other_countries_list = other_value_list(country_value_count ,'country',optimal_value_count_country)
print("Countries to be replaced with Other : ", len(other_countries_list))

Countries to be replaced with Other :  196


#### Function to replace the list with 'Other'

In [69]:
def replace_countries_with_other(row_list):
    for df_country in row_list:
        if df_country in other_countries_list:
            row_list[row_list.index(df_country)] = 'Other'
    return row_list

#### Replace all the other countries with 'Other'

In [70]:
# Replace all the other countries with 'Other'
toBeExplodeddf['country'] = toBeExplodeddf['country'].apply(replace_countries_with_other)

#### Explode the country to perform a value_count. Helps us understand the amount of features we would be adding.

In [71]:
# Explode the country to perform a value_count. Helps us understand the amount of features we would be adding.

replaced_country_value_count = value_counts_from_list_column(toBeExplodeddf,'country')
print("Number of unique Countries after replacing with Other : ", len(replaced_country_value_count))

replaced_country_value_count.head(10)

Number of unique Countries after replacing with Other :  25


,country,frequency
23,USA,225843
15,None,129203
16,Other,61778
22,UK,46742
6,France,22611
7,Germany,17978
3,Canada,17098
12,Japan,15440
10,India,15033
11,Italy,11580


#### One Hot encoding of all category value based classes

In [72]:
# One Hot encoding of all category value based classes

print("\nDataframe Shape Before One Hot encoding of country: ",str(toBeExplodeddf.shape))
toBeExplodeddf = one_hot_encode_and_merge_orginalDF(toBeExplodeddf,columnName ='country',renameCol = 'country_')
print("\nDataframe Shape After One Hot encoding of country: ",str(toBeExplodeddf.shape))

toBeExplodeddf.head()


Dataframe Shape Before One Hot encoding of country:  (621943, 42)

Dataframe Shape After One Hot encoding of country:  (621943, 66)


,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,country_None,country_Other,country_Russia,country_Soviet Union,country_Spain,country_Sweden,country_Turkey,country_UK,country_USA,country_West Germany
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,0,0,0,0,0,0,0,0,1,0
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,None,None,6.5,148,1892-10-28,5min,1892,...,0,0,0,0,0,0,0,0,0,0
2,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,4min,1892,...,0,0,0,0,0,0,0,0,0,0
3,tt0000004,Egy jó pohár sör (1892) - IMDb,1,None,None,6.6,90,1892-10-28,None,1892,...,0,0,0,0,0,0,0,0,0,0
4,tt0000005,A patkolókovács (1893) - IMDb,18,3,None,6.2,1499,1893-05-09,1min,1893,...,0,0,0,0,0,0,0,0,1,0


In [73]:
toBeExplodeddf.columns

Index(['movie_id', 'title', 'review_count_user', 'review_count_critic',
       'metascore', 'rating_value', 'rating_count', 'date_published',
       'duration', 'title_year', 'genre_tags', 'director_names',
       'director_ids', 'language', 'release_date', 'runtime', 'directed_by',
       'release_year', 'movie_year', 'opening_weekend_value',
       'open_wk_cur_AUD', 'open_wk_cur_CAD', 'open_wk_cur_DEM',
       'open_wk_cur_GBP', 'open_wk_cur_ITL', 'open_wk_cur_JPY',
       'open_wk_cur_SEK', 'open_wk_cur_SGD', 'open_wk_cur_USD', 'gross_value',
       'gross_cur_USD', 'budget_value', 'budget_cur_AUD', 'budget_cur_CAD',
       'budget_cur_EUR', 'budget_cur_GBP', 'budget_cur_INR', 'budget_cur_USD',
       'color_Black and White', 'color_Color', 'color_None',
       'country_Argentina', 'country_Australia', 'country_Brazil',
       'country_Canada', 'country_Denmark', 'country_Finland',
       'country_France', 'country_Germany', 'country_Greece',
       'country_Hong Kong', 'country_In

## 6) Language - Data cleaning & Pre processing

### 6a) Data cleaning

#### Verifying the count of improper records - Before

In [74]:
# Verifying

print("Rows containing Also Known as in language column before processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['language'].str.contains(pat = 'Also Known As',na=False)]))

print("Rows containing See more in language column before processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['language'].str.contains(pat = 'See more',na=False)]))

print("Rows containing Numbers in language column before processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['language'].str.contains(pat = '\d+',na=False,)]))

Rows containing Also Known as in language column before processing: 9
Rows containing See more in language column before processing: 559
Rows containing Numbers in language column before processing: 512


#### Cleaning the improper records based on the pattern

In [75]:
# Pattern : Also Known as & See more. Clear the ones with texts for language Column

toBeExplodeddf.loc[toBeExplodeddf['language'].str.contains(pat = 'Also Known As',na=False),'language'] = "Other"
toBeExplodeddf.loc[toBeExplodeddf['language'].str.contains(pat = 'See more',na=False),'language'] = "Other"
toBeExplodeddf.loc[toBeExplodeddf['language'].str.contains(pat = '\d+',na=False),'language'] = "Other"

#### Verifying the count of improper records - After cleaning

In [76]:
# Count of records which match the improper pattern

print("\nRows containing Also Known as in language column after processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['language'].str.contains(pat = 'Also Known As',na=False)]))

print("Rows containing See more in language column after processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['language'].str.contains(pat = 'See more',na=False)]))

print("Rows containing Numbers in language column before processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['language'].str.contains(pat = '\d+',na=False,)]))

# print("Rows containing something in brackets in Language column before processing:", 
#       len(toBeExplodeddf.loc[toBeExplodeddf['Language'].str.contains(pat = '\((.*?)\)',na=False,)]))


Rows containing Also Known as in language column after processing: 0
Rows containing See more in language column after processing: 0
Rows containing Numbers in language column before processing: 0


### 6b) Pre processing - Optimal Value & One hot encoding

#### Split cells into lists

In [77]:
toBeExplodeddf['language'] = toBeExplodeddf['language'].str.split("|", expand = False)

#### Strip spaces from the list

In [78]:
toBeExplodeddf['language'] = toBeExplodeddf['language'].apply(strip_spaces_from_list)
toBeExplodeddf['language'].head()

0    [None]
1    [None]
2    [None]
3    [None]
4    [None]
Name: language, dtype: object

#### Value count among a list column

In [79]:
language_value_count = value_counts_from_list_column(toBeExplodeddf,'language')
print("Number of unique Languages : ", len(language_value_count))

Number of unique Languages :  311


#### Optimal value count for Languages

In [80]:
required_percent = 90
optimal_value_count_language = find_optimal_value_count(language_value_count['frequency'],required_percent)
print("The amount of languages required to reach " + str(required_percent) + 
      " percentage is " + str(optimal_value_count_language) + " languages")

The amount of languages required to reach 90 percentage is 16 languages


#### A list of Languages which are not in the optimal value count length replaced with 'Other'

In [81]:
other_languages_list = other_value_list(language_value_count ,'language',optimal_value_count_language)
print("Languages to be replaced with Other : ", len(other_languages_list))

Languages to be replaced with Other :  295


#### Function to replace languages with other

In [82]:
def replace_language_with_other(row_list):
    for df_language in row_list:
        if df_language in other_languages_list:
            row_list[row_list.index(df_language)] = 'Other'
    return row_list

In [83]:
# Replace all the other langues with 'Other'
toBeExplodeddf['language'] = toBeExplodeddf['language'].apply(replace_language_with_other)

#### Explode the Languages to perform a value_count. Helps us understand the amount of features we would be adding.

In [84]:
# Explode the Languages to perform a value_count. Helps us understand the amount of features we would be adding.

replaced_language_value_count = value_counts_from_list_column(toBeExplodeddf,'language')
print("Number of unique Languages after replacing with Other : ", len(replaced_language_value_count))

Number of unique Languages after replacing with Other :  17


In [85]:
replaced_language_value_count.head(10)

,language,frequency
2,English,343761
10,None,108592
11,Other,64380
4,German,27262
3,French,25770
14,Spanish,24377
8,Japanese,15833
7,Italian,12352
13,Russian,9974
6,Hindi,7050


#### One Hot encoding of all category value based classes

In [86]:
# One Hot encoding of all category value based classes

print("\nDataframe Shape Before One Hot encoding of Language: ",str(toBeExplodeddf.shape))
toBeExplodeddf = one_hot_encode_and_merge_orginalDF(toBeExplodeddf,columnName ='language',renameCol = 'language_')
print("\nDataframe Shape After One Hot encoding of Language: ",str(toBeExplodeddf.shape))


Dataframe Shape Before One Hot encoding of Language:  (621943, 66)

Dataframe Shape After One Hot encoding of Language:  (621943, 82)


In [87]:
toBeExplodeddf.head()

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,language_Italian,language_Japanese,language_Mandarin,language_None,language_Other,language_Portuguese,language_Russian,language_Spanish,language_Swedish,language_Turkish
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,0,0,0,1,0,0,0,0,0,0
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,None,None,6.5,148,1892-10-28,5min,1892,...,0,0,0,1,0,0,0,0,0,0
2,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,4min,1892,...,0,0,0,1,0,0,0,0,0,0
3,tt0000004,Egy jó pohár sör (1892) - IMDb,1,None,None,6.6,90,1892-10-28,None,1892,...,0,0,0,1,0,0,0,0,0,0
4,tt0000005,A patkolókovács (1893) - IMDb,18,3,None,6.2,1499,1893-05-09,1min,1893,...,0,0,0,1,0,0,0,0,0,0


In [88]:
toBeExplodeddf.columns

Index(['movie_id', 'title', 'review_count_user', 'review_count_critic',
       'metascore', 'rating_value', 'rating_count', 'date_published',
       'duration', 'title_year', 'genre_tags', 'director_names',
       'director_ids', 'release_date', 'runtime', 'directed_by',
       'release_year', 'movie_year', 'opening_weekend_value',
       'open_wk_cur_AUD', 'open_wk_cur_CAD', 'open_wk_cur_DEM',
       'open_wk_cur_GBP', 'open_wk_cur_ITL', 'open_wk_cur_JPY',
       'open_wk_cur_SEK', 'open_wk_cur_SGD', 'open_wk_cur_USD', 'gross_value',
       'gross_cur_USD', 'budget_value', 'budget_cur_AUD', 'budget_cur_CAD',
       'budget_cur_EUR', 'budget_cur_GBP', 'budget_cur_INR', 'budget_cur_USD',
       'color_Black and White', 'color_Color', 'color_None',
       'country_Argentina', 'country_Australia', 'country_Brazil',
       'country_Canada', 'country_Denmark', 'country_Finland',
       'country_France', 'country_Germany', 'country_Greece',
       'country_Hong Kong', 'country_India', 'count

## 7) Genre Tags - Data Cleaning & Pre processing

### Split cells into lists

In [89]:
toBeExplodeddf['genre_tags'] = toBeExplodeddf['genre_tags'].str.split("\t", expand = False)

### 7a) Pre processing - Genre tags

In [90]:
toBeExplodeddf['genre_tags'] = toBeExplodeddf['genre_tags'].apply(strip_spaces_from_list)
toBeExplodeddf['genre_tags'].head()

0                   [Documentary, Short]
1                     [Animation, Short]
2    [Animation, Comedy, Short, Romance]
3                     [Animation, Short]
4                                [Short]
Name: genre_tags, dtype: object

In [91]:
genre_value_count = value_counts_from_list_column(toBeExplodeddf,'genre_tags')
print("Number of unique Genre : ", len(genre_value_count))

Number of unique Genre :  29


In [92]:
# genre_value_count[genre_value_count['genre_tags'].str.contains('None')]
genre_value_count.head(10)

,genre_tags,frequency
8,Drama,234482
5,Comedy,207092
23,Short,97059
6,Crime,67394
0,Action,67109
21,Romance,64400
7,Documentary,63204
9,Family,63188
26,Thriller,60012
3,Animation,57348


In [93]:
required_percent = 90
optimal_value_count_genre = find_optimal_value_count(genre_value_count['frequency'],required_percent)
print("The amount of languages required to reach " + str(required_percent) + 
      " percentage is " + str(optimal_value_count_genre) + " genre")

The amount of languages required to reach 90 percentage is 16 genre


In [94]:
# One Hot encoding of all category value based classes

print("\nDataframe Shape Before One Hot encoding of Genre tags: ",str(toBeExplodeddf.shape))
toBeExplodeddf = one_hot_encode_and_merge_orginalDF(toBeExplodeddf,columnName ='genre_tags',renameCol = 'genre_')
print("\nDataframe Shape After One Hot encoding of Genre tags : ",str(toBeExplodeddf.shape))


Dataframe Shape Before One Hot encoding of Genre tags:  (621943, 82)

Dataframe Shape After One Hot encoding of Genre tags :  (621943, 110)


In [95]:
toBeExplodeddf.columns

Index(['movie_id', 'title', 'review_count_user', 'review_count_critic',
       'metascore', 'rating_value', 'rating_count', 'date_published',
       'duration', 'title_year',
       ...
       'genre_None', 'genre_Reality-TV', 'genre_Romance', 'genre_Sci-Fi',
       'genre_Short', 'genre_Sport', 'genre_Talk-Show', 'genre_Thriller',
       'genre_War', 'genre_Western'],
      dtype='object', length=110)

## 8) Runtime - Data Cleaning & Pre processing

##### Split runtime column and obtain the corresponding minutes

In [96]:
replace_runtime_string = { 'min' : '', ',' : '','None': np.nan}

In [97]:
# Split by | and take only the fist occurence of the array which contains our interested value
# Split by ( and take only the fist occurence of the array which contains minutes
# Strip the values
# Replace min , comma and None

toBeExplodeddf['runtime'] = toBeExplodeddf['runtime'].str.split(pat = '|', n = 1, expand = False).str[0] \
                                                     .str.split(pat = '(', n = 1, expand = False).str[0] \
                                                     .str.strip() \
                                                     .replace(replace_runtime_string, regex=True)

#### Verifying the count of improper records - Before

In [98]:
# Verifying

print("Rows containing alphabets in runtime column before processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['runtime'].str.contains(pat = '[A-Za-z]',na=False)]))

Rows containing alphabets in runtime column before processing: 3


#### Cleaning the improper records based on the pattern

In [99]:
# Pattern :Trivia. Clear the ones with texts for runtime Column

toBeExplodeddf.loc[toBeExplodeddf['runtime'].str.contains(pat = '[A-Za-z]',na=False),'runtime'] = np.nan


#### Verifying the count of improper records - after

In [100]:
# Verifying

print("Rows containing Trivia in runtime column after processing:", 
      len(toBeExplodeddf.loc[toBeExplodeddf['runtime'].str.contains(pat = '[A-Za-z]',na=False)]))

Rows containing Trivia in runtime column after processing: 0


## 9) Review Count user and Review Count critic - Data Cleaning & Preprocessing

In [101]:
# Before Cleaning Review count user
toBeExplodeddf.review_count_user.value_counts(dropna = False).nlargest(15)

None       336075
1           88176
2           37233
1critic     29516
3           21013
4           13893
5            9685
2critic      8323
6            7094
7            5609
8            4438
9            3785
3critic      3570
0            3487
10           3109
Name: review_count_user, dtype: int64

In [102]:
# Before Cleaning Review count Critic
toBeExplodeddf.review_count_critic.value_counts(dropna = False).nlargest(15)

None    499710
1        34235
2        16927
3        10522
4         7751
5         5769
6         4434
7         3737
8         3004
9         2536
10        2191
11        1899
12        1657
13        1512
14        1392
Name: review_count_critic, dtype: int64

#### Function to clean the values in the two columns : 
1. Remove critic
2. replace None with Nan
3. Moving values incorrectly placed to review count critic

In [103]:
def clean_review_count_user_and_critic(df):
    cond = df['review_count_user'].str.contains('critic',na=False)
    print("The number of values misplaced from critic column to user column = ", str(cond.sum()))
    
    # Data cleaning of review_count_critic column where the rows contain the word critic
    print("\nCopying the records which were supposed to be in critic column from the user column . . .")
    df.loc[cond,'review_count_critic'] = df['review_count_user']

    # Remove the data containing the word 'critic' in review_user_count and replace it with NaN ( Check if you want to replace with zero )
    print("\nRemoving the records which were supposed to be in critic column from the user column . . .")
    df.loc[(df['review_count_user'].str.contains('critic',na=False)), 'review_count_user'] = None

    # Remove the word None in review_count_user and replace it with NaN ( Check if you want to replace with zero)
    print("\nReplacing all the None value records to NaN . . .")
    df['review_count_user']  = df['review_count_user'].replace(to_replace='None', value=np.nan)
    
    # Extract only the number from the column values in review_count critic. ie. omitting the word 'critic'
    print("\nSince the data had some issues while parsing, cleaning the records which were moved from the user column as they contained the word 'critic' . . .")
    df.review_count_critic = df.review_count_critic.str.extract('(\d+)')
    
    # # Fill NA with 0
    df['review_count_user'].fillna(0, inplace=True)
    df['review_count_critic'].fillna(0, inplace=True)

In [104]:
clean_review_count_user_and_critic(toBeExplodeddf)

The number of values misplaced from critic column to user column =  48201

Copying the records which were supposed to be in critic column from the user column . . .

Removing the records which were supposed to be in critic column from the user column . . .

Replacing all the None value records to NaN . . .

Since the data had some issues while parsing, cleaning the records which were moved from the user column as they contained the word 'critic' . . .


In [105]:
print("\nDataframe Shape after cleaning : ",str(toBeExplodeddf.shape))


Dataframe Shape after cleaning :  (621943, 110)


In [106]:
# After Cleaning Review count user
toBeExplodeddf.review_count_user.value_counts(dropna = False).nlargest(15)

0     384276
1      88176
2      37233
3      21013
4      13893
5       9685
6       7094
7       5609
8       4438
9       3785
0       3487
10      3109
11      2631
12      2338
13      2067
Name: review_count_user, dtype: int64

In [107]:
# After  Cleaning Review count Critic
toBeExplodeddf.review_count_critic.value_counts(dropna = False).nlargest(15)

0     451509
1      63751
2      25250
3      14092
4       9745
5       7083
6       5273
7       4358
8       3434
9       2882
10      2473
11      2075
12      1802
13      1620
14      1482
Name: review_count_critic, dtype: int64

## 10) Generating new column Features : TV or Movie episode

In [108]:
toBeExplodeddf.title.str.contains('\(TV ', na=False, regex=True).sum()


312418

In [109]:
toBeExplodeddf.title.str.contains('\(TV Episode', na=False, regex=True).sum() + \
toBeExplodeddf.title.str.contains('\(TV Movie', na=False, regex=True).sum() + \
toBeExplodeddf.title.str.contains('\(TV Series', na=False, regex=True).sum() + \
toBeExplodeddf.title.str.contains('\(TV Mini', na=False, regex=True).sum() + \
toBeExplodeddf.title.str.contains('\(TV Short', na=False, regex=True).sum()

312418

In [110]:
toBeExplodeddf.title.str.contains('\(Video ', na=False, regex=True).sum()


24581

In [111]:
toBeExplodeddf.title.str.contains('\(Video Game', na=False, regex=True).sum()

3770

### Column feature Title category and subtitle category

In [112]:
# Create a new column indiciating whether the record is TV Episode or Movie

toBeExplodeddf['title_category'] = np.nan

toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(TV Episode', na=False, regex=True),'title_category'] = 'TV'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(TV Movie', na=False, regex=True),'title_category'] = 'TV'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(TV Series', na=False, regex=True),'title_category'] = 'TV'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(TV Mini', na=False, regex=True),'title_category'] = 'TV'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(TV Short', na=False, regex=True),'title_category'] = 'TV'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(Video Game', na=False, regex=True),'title_category'] = 'video game'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(Video ', na=False, regex=True),'title_category'] = 'video'
toBeExplodeddf['title_category']  = toBeExplodeddf['title_category'].fillna('movie')

In [113]:
# Create a new column indiciating whether the record is TV Episode or Movie

toBeExplodeddf['title_subcategory'] = np.nan

toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(TV Episode', na=False, regex=True),'title_subcategory'] = 'TV episode'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(TV Movie', na=False, regex=True),'title_subcategory'] = 'TV movie'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(TV Series', na=False, regex=True),'title_subcategory'] = 'TV series'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(TV Mini', na=False, regex=True),'title_subcategory'] = 'TV miniseries'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(TV Short', na=False, regex=True),'title_subcategory'] = 'TV short'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(Video Game', na=False, regex=True),'title_subcategory'] = 'game'
toBeExplodeddf.loc[toBeExplodeddf.title.str.contains('\(Video ', na=False, regex=True),'title_subcategory'] = 'video'
toBeExplodeddf['title_subcategory']  = toBeExplodeddf['title_subcategory'].fillna('movie')

In [114]:
toBeExplodeddf['title_category'].value_counts()

TV       312418
movie    284944
video     24581
Name: title_category, dtype: int64

In [115]:
toBeExplodeddf['title_subcategory'].value_counts()

movie            284944
TV episode       227376
TV series         40810
TV movie          36448
video             24581
TV miniseries      5892
TV short           1892
Name: title_subcategory, dtype: int64

#### one hot encoding of Title Category and title subcategory

In [116]:
# one hot encoding of title category and title subcategory

print("\nDataframe Shape Before One hot Encoding title_category : ",str(toBeExplodeddf.shape))

toBeExplodeddf = pd.concat([toBeExplodeddf, pd.get_dummies(toBeExplodeddf['title_category'],
                                                           prefix='title_category')],axis=1)

toBeExplodeddf = pd.concat([toBeExplodeddf, pd.get_dummies(toBeExplodeddf['title_subcategory'],
                                                           prefix='title_subcategory')],axis=1)

toBeExplodeddf.drop(['title_category','title_subcategory'],axis=1, inplace=True)

print("\nDataframe Shape After One hot Encoding title_category : ",str(toBeExplodeddf.shape))



Dataframe Shape Before One hot Encoding title_category :  (621943, 112)

Dataframe Shape After One hot Encoding title_category :  (621943, 120)


#### Bring back to one single column. Reverse One hot encoding

In [117]:
# In case we want to bring back the one hot encoding to normal columns , to switch back to single column

# toBeExplodeddf.loc[:, toBeExplodeddf.columns.str.startswith('title_category')].idxmax(axis = 'columns')
# toBeExplodeddf.loc[:, toBeExplodeddf.columns.str.startswith('title_subcategory')].idxmax(axis = 'columns')

In [118]:
print("\nDataframe Shape after creating a new column : ",str(toBeExplodeddf.shape))


Dataframe Shape after creating a new column :  (621943, 120)


In [119]:
toBeExplodeddf.head(3)

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,title_category_TV,title_category_movie,title_category_video,title_subcategory_TV episode,title_subcategory_TV miniseries,title_subcategory_TV movie,title_subcategory_TV series,title_subcategory_TV short,title_subcategory_movie,title_subcategory_video
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,0,1,0,0,0,0,0,0,1,0
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,0,None,6.5,148,1892-10-28,5min,1892,...,0,1,0,0,0,0,0,0,1,0
2,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,4min,1892,...,0,1,0,0,0,0,0,0,1,0


## 11) Rating value - Data Cleaning

In [120]:
# Rating count and rating value are being removed where occurrence is None

dropRatingLessRecords = toBeExplodeddf.loc[toBeExplodeddf['rating_value'] == 'None']
len(dropRatingLessRecords)

3968

In [121]:
toBeExplodeddf.drop(dropRatingLessRecords.index, inplace= True)

In [122]:
toBeExplodeddf[toBeExplodeddf.rating_count.str.contains('None')]

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,title_category_TV,title_category_movie,title_category_video,title_subcategory_TV episode,title_subcategory_TV miniseries,title_subcategory_TV movie,title_subcategory_TV series,title_subcategory_TV short,title_subcategory_movie,title_subcategory_video


In [123]:
toBeExplodeddf.shape

(617975, 120)

## 12) Director IDs and Director Names : Data cleaning and  Pre processing

In [124]:
# toBeExplodeddf[toBeExplodeddf.movie_id.str.contains('tt0067806')]

In [125]:
# To split multiple entries and save as list for director ID and Names column.

toBeExplodeddf['director_ids'] = toBeExplodeddf['director_ids'].str.split("\t", expand = False)
toBeExplodeddf['director_names'] = toBeExplodeddf['director_names'].str.split("\t", expand = False)

In [126]:
# Function to identify equal list items in Director names and ID column. Appends 'None' to the ones which are not equal

def equalListElements(df):
    #if (len(df['director_ids']) - len(df['director_names'])) > 0 :
    return (df['director_names'] + ['None'] * (len(df['director_ids']) - len(df['director_names'])))

In [127]:
# Equal number of elements in director ID and director Names to perform explode in the next step

toBeExplodeddf['equalizing_director_names'] = toBeExplodeddf.apply(equalListElements, axis='columns')

In [128]:
toBeExplodeddf.head(2)

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,title_category_movie,title_category_video,title_subcategory_TV episode,title_subcategory_TV miniseries,title_subcategory_TV movie,title_subcategory_TV series,title_subcategory_TV short,title_subcategory_movie,title_subcategory_video,equalizing_director_names
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,1,0,0,0,0,0,0,1,0,[William K.L. Dickson]
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,0,None,6.5,148,1892-10-28,5min,1892,...,1,0,0,0,0,0,0,1,0,[Émile Reynaud]


In [129]:
# Number of total elements in the list column of director_ids 
sum(map(len, toBeExplodeddf.director_ids))

798873

In [130]:
# Number of total elements in the list column of new director_names 
sum(map(len, toBeExplodeddf.equalizing_director_names))

798873

# Explode data

In [131]:
# Unnest/explode function which splits the lists to multiple row entries

def unnesting(df, explode):
    idx = df.index.repeat(df[explode[0]].str.len())
    df1 = pd.concat([
        pd.DataFrame({x: np.concatenate(df[x].values)}) for x in explode], axis=1)
        #pd.DataFrame({x: np.array(df[x].values)}) for x in explode], axis=1)
    df1.index = idx
    return df1.join(df.drop(explode, 1), how='left')

In [132]:
# Explode/Unnest the director_id and new_director_names column
unnestedDF = unnesting(toBeExplodeddf,['director_ids','equalizing_director_names']).reset_index(drop=True)
print("Unnested dataframe Shape after exploding/unnesting : ",str(unnestedDF.shape))

Unnested dataframe Shape after exploding/unnesting :  (798873, 121)


In [133]:
# Drop the old director_names and rename the equal_director_names as director_names
unnestedDF = unnestedDF.drop(columns=['director_names'])
unnestedDF.rename(columns={'equalizing_director_names':'director_names'}, inplace=True)

In [134]:
unnestedDF['director_ids'] = unnestedDF.director_ids.str.strip()
unnestedDF['director_names'] = unnestedDF.director_names.str.strip()

In [135]:
print("Unnested dataframe Shape : ",str(unnestedDF.shape))
unnestedDF.head()

Unnested dataframe Shape :  (798873, 120)


,director_ids,director_names,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,...,title_category_TV,title_category_movie,title_category_video,title_subcategory_TV episode,title_subcategory_TV miniseries,title_subcategory_TV movie,title_subcategory_TV series,title_subcategory_TV short,title_subcategory_movie,title_subcategory_video
0,nm0005690,William K.L. Dickson,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,...,0,1,0,0,0,0,0,0,1,0
1,nm0721526,Émile Reynaud,tt0000002,A bohóc és kutyái (1892) - IMDb,0,0,None,6.5,148,1892-10-28,...,0,1,0,0,0,0,0,0,1,0
2,nm0721526,Émile Reynaud,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,...,0,1,0,0,0,0,0,0,1,0
3,nm0721526,Émile Reynaud,tt0000004,Egy jó pohár sör (1892) - IMDb,1,0,None,6.6,90,1892-10-28,...,0,1,0,0,0,0,0,0,1,0
4,nm0005690,William K.L. Dickson,tt0000005,A patkolókovács (1893) - IMDb,18,3,None,6.2,1499,1893-05-09,...,0,1,0,0,0,0,0,0,1,0


In [136]:
# Records having director_names empty
unnestedDF.loc[unnestedDF.director_names.str.contains('None')]

,director_ids,director_names,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,...,title_category_TV,title_category_movie,title_category_video,title_subcategory_TV episode,title_subcategory_TV miniseries,title_subcategory_TV movie,title_subcategory_TV series,title_subcategory_TV short,title_subcategory_movie,title_subcategory_video
155,nm0698645,None,tt0000162,A Niagara-vízesés (1897) - IMDb,8,1,None,5.6,313,2000-10-11,...,0,1,0,0,0,0,0,0,1,0
387,nm0567363,None,tt0000680,Falsely Accused! (1908) - IMDb,1,0,None,5.9,10,1908-01-18,...,0,1,0,0,0,0,0,0,1,0
1090,nm0091767,None,tt0003355,Saved by the Pony Express (1911) - IMDb,1,0,None,4.8,6,1911-08-01,...,0,1,0,0,0,0,0,0,1,0
1094,nm0596218,None,tt0003386,Sodoms Ende (1913) - IMDb,0,0,None,8.5,8,None,...,0,1,0,0,0,0,0,0,1,0
4712,nm0425766,None,tt0019385,Simba: The King of the Beasts (1928) - IMDb,2,1,None,6.9,17,1928-01-25,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798846,nm8245192,None,tt6697116,The Grindhouse Radio (TV Series 2015– ) - IMDb,0,0,None,9.5,15,None,...,1,0,0,0,0,0,1,0,0,0
798847,nm10030840,None,tt6697116,The Grindhouse Radio (TV Series 2015– ) - IMDb,0,0,None,9.5,15,None,...,1,0,0,0,0,0,1,0,0,0
798851,nm8881418,None,tt6705808,Gunnars Kjeller (TV Series 2016– ) - IMDb,0,0,None,8.9,17,None,...,1,0,0,0,0,0,1,0,0,0
798852,nm8881417,None,tt6705808,Gunnars Kjeller (TV Series 2016– ) - IMDb,0,0,None,8.9,17,None,...,1,0,0,0,0,0,1,0,0,0


In [137]:
unnestedDF.loc[unnestedDF.director_names.str.contains('None')].count()

director_ids                   184663
director_names                 184663
movie_id                       184663
title                          184663
review_count_user              184663
                                ...  
title_subcategory_TV movie     184663
title_subcategory_TV series    184663
title_subcategory_TV short     184663
title_subcategory_movie        184663
title_subcategory_video        184663
Length: 120, dtype: int64

## Director IDs existed for the missing director names.
### Import IMDB Name data set and merge with the exploded dataframe

In [138]:
IMDBNameDatafileLocation = 'C:\\Yuva\\ITU\\3rd Sem\\Research Topics\\Movies & Gender\\Website - IMDB\\name\\data.tsv'

In [139]:
# Import as IMDB Dataset Name Dataframe

IMDBNameDF = pd.read_csv(IMDBNameDatafileLocation, sep='\t')

In [140]:
IMDBNameDF.rename(columns={'nconst':'director_ids'}, inplace=True)

In [141]:
IMDBNameDF.shape

(10008408, 6)

In [142]:
IMDBNameDF.head()

,director_ids,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0043044"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0037382,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,producer","tt0057345,tt0059956,tt0049189,tt0054452,tt0054..."
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0077975,tt0078723,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0050986,tt0050976,tt0083922"


In [143]:
# Merge the IMDBName data with our exploded data to fill the missing director_names
print("\nunnestedDF dataframe Shape Before merge : ",str(unnestedDF.shape))
explodedAndMergeDF = unnestedDF.merge(IMDBNameDF, on='director_ids', how ='left')
print("\nexplodedAndMergeDF dataframe Shape After merge : ",str(explodedAndMergeDF.shape))


unnestedDF dataframe Shape Before merge :  (798873, 120)

explodedAndMergeDF dataframe Shape After merge :  (798873, 125)


In [144]:
len(explodedAndMergeDF.loc[explodedAndMergeDF['director_names'].str.contains('None'),'director_names'])

184663

In [145]:
# Replace all the missing director names from the primaryName column of Original IMDB data tsv
explodedAndMergeDF.loc[explodedAndMergeDF['director_names'].str.contains('None'),'director_names'] = explodedAndMergeDF['primaryName']

In [146]:
explodedAndMergeDF['director_names'].isna().sum()

12

In [147]:
# Drop the merged director_names and Directed_by
print("\nexplodedAndMergeDF dataframe Shape  : ",str(explodedAndMergeDF.shape))


explodedAndMergeDF dataframe Shape  :  (798873, 125)


## 13) Generate 3 new features indicating if the profession of the director was primary, secondary or tertiary

In [148]:
# primary profession as a director

explodedAndMergeDF['director_profession_primary'] = explodedAndMergeDF['primaryProfession'] \
                                                    .str.split(pat = ',', n = 1 , expand = False).str[0] \
                                                    .str.strip() \
                                                    .replace({'director':1})

explodedAndMergeDF.loc[~(explodedAndMergeDF['director_profession_primary'] == 1),'director_profession_primary'] = 0

In [149]:
# Secondary profession as a director

explodedAndMergeDF['director_profession_secondary'] = explodedAndMergeDF['primaryProfession'] \
                                                    .str.split(pat = ',', n = 2 , expand = False).str[1] \
                                                    .str.strip() \
                                                    .replace({'director':1})

explodedAndMergeDF.loc[~(explodedAndMergeDF['director_profession_secondary'] == 1),'director_profession_secondary'] = 0

In [150]:
# Tertiary profession as a director

explodedAndMergeDF['director_profession_tertiary'] = explodedAndMergeDF['primaryProfession'] \
                                                    .str.split(pat = ',', n = 3 , expand = False).str[2] \
                                                    .str.strip() \
                                                    .replace({'director':1})

explodedAndMergeDF.loc[~(explodedAndMergeDF['director_profession_tertiary'] == 1),'director_profession_tertiary'] = 0

In [151]:
explodedAndMergeDF.head()

,director_ids,director_names,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,...,title_subcategory_movie,title_subcategory_video,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,director_profession_primary,director_profession_secondary,director_profession_tertiary
0,nm0005690,William K.L. Dickson,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,...,1,0,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt0219560,tt1496763,tt0308254,tt1428455",0,1,0
1,nm0721526,Émile Reynaud,tt0000002,A bohóc és kutyái (1892) - IMDb,0,0,None,6.5,148,1892-10-28,...,1,0,Émile Reynaud,1844,1918,director,"tt0413219,tt2184231,tt2184201,tt0000003,tt0000...",1,0,0
2,nm0721526,Émile Reynaud,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,...,1,0,Émile Reynaud,1844,1918,director,"tt0413219,tt2184231,tt2184201,tt0000003,tt0000...",1,0,0
3,nm0721526,Émile Reynaud,tt0000004,Egy jó pohár sör (1892) - IMDb,1,0,None,6.6,90,1892-10-28,...,1,0,Émile Reynaud,1844,1918,director,"tt0413219,tt2184231,tt2184201,tt0000003,tt0000...",1,0,0
4,nm0005690,William K.L. Dickson,tt0000005,A patkolókovács (1893) - IMDb,18,3,None,6.2,1499,1893-05-09,...,1,0,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt0219560,tt1496763,tt0308254,tt1428455",0,1,0


## 14) Generate Features from Known for titles column - Count & average rating value

In [152]:
# #### Dictionary for lookup of values present in our dataset in the corresponding known titles column

# # Movie ID  as the key and the corresponding movie number as value
# movie_id_to_movie_number_dict = dict(zip(explodedAndMergeDF.movie_id, explodedAndMergeDF.movie_id.str.extract('(\d+)')[0]))

# #### Dictionary for lookup of the corresponding rating value from the movie number

# movie_number_to_rating_value_dict = dict(zip(explodedAndMergeDF.movie_id.str.extract('(\d+)')[0], explodedAndMergeDF.rating_value))

#### Dictionary for lookup of the movie_id to rating_value

In [153]:
rating_value_dict = dict(zip(explodedAndMergeDF.movie_id, explodedAndMergeDF.rating_value))

#### Split known for titles to lists

In [154]:
knownForTitlesDF = explodedAndMergeDF['knownForTitles'].str.split(pat = ',',expand = True)

In [155]:
knownForTitlesDF.head()

,0,1,2,3,4,5,6,7,8,9
0,tt0219560,tt1496763,tt0308254,tt1428455,None,None,None,None,None,None
1,tt0413219,tt2184231,tt2184201,tt0000003,tt0000003,tt2184201,tt2184231,tt0413219,None,None
2,tt0413219,tt2184231,tt2184201,tt0000003,tt0000003,tt2184201,tt2184231,tt0413219,None,None
3,tt0413219,tt2184231,tt2184201,tt0000003,tt0000003,tt2184201,tt2184231,tt0413219,None,None
4,tt0219560,tt1496763,tt0308254,tt1428455,None,None,None,None,None,None


In [156]:
# Null values in each row before removing the movie id's that doesn't exist in our dataset
knownForTitlesDF.isna().sum()

0      3336
1     16894
2     34712
3     46829
4    522287
5    522290
6    529261
7    529261
8    798870
9    798870
dtype: int64

In [157]:
print("Count of known title records for the director before removing the movie titles which are unavailable or unknown to our dataset")
knownForTitlesDF.count(axis = 'columns')

Count of known title records for the director before removing the movie titles which are unavailable or unknown to our dataset


0         4
1         8
2         8
3         8
4         4
         ..
798868    8
798869    4
798870    1
798871    4
798872    2
Length: 798873, dtype: int64

In [158]:
for i in knownForTitlesDF.columns:
    knownForTitlesDF[i] = knownForTitlesDF[i].map(rating_value_dict).astype(float)

In [159]:
print("Count of known title records for the director after removing the movie titles which are unavailable or unknown to our dataset")
knownForTitlesDF.count(axis = 'columns')

Count of known title records for the director after removing the movie titles which are unavailable or unknown to our dataset


0         4
1         6
2         6
3         6
4         4
         ..
798868    2
798869    1
798870    1
798871    3
798872    2
Length: 798873, dtype: int64

In [160]:
knownForTitlesDF.head()

,0,1,2,3,4,5,6,7,8,9
0,5.9,5.0,6.6,5.5,NaN,NaN,NaN,NaN,NaN,NaN
1,6.2,NaN,5.7,6.6,6.6,5.7,NaN,6.2,NaN,NaN
2,6.2,NaN,5.7,6.6,6.6,5.7,NaN,6.2,NaN,NaN
3,6.2,NaN,5.7,6.6,6.6,5.7,NaN,6.2,NaN,NaN
4,5.9,5.0,6.6,5.5,NaN,NaN,NaN,NaN,NaN,NaN


#### Feature Column: known title count and average rating value of known titles

In [161]:
# Feature Column: known title count and average rating value known titles

explodedAndMergeDF['director_known_titles_count'] = knownForTitlesDF.count(axis = 'columns')
explodedAndMergeDF['director_known_titles_average_rating_value'] =  round(knownForTitlesDF.sum(axis = 'columns' , skipna= True) / knownForTitlesDF.count(axis = 'columns'),2)

# End

In [162]:
explodedAndMergeDF.head()
#true = explodedAndMergeDF.copy()

,director_ids,director_names,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,...,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,director_profession_primary,director_profession_secondary,director_profession_tertiary,director_known_titles_count,director_known_titles_average_rating_value
0,nm0005690,William K.L. Dickson,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,...,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt0219560,tt1496763,tt0308254,tt1428455",0,1,0,4,5.75
1,nm0721526,Émile Reynaud,tt0000002,A bohóc és kutyái (1892) - IMDb,0,0,None,6.5,148,1892-10-28,...,Émile Reynaud,1844,1918,director,"tt0413219,tt2184231,tt2184201,tt0000003,tt0000...",1,0,0,6,6.17
2,nm0721526,Émile Reynaud,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,...,Émile Reynaud,1844,1918,director,"tt0413219,tt2184231,tt2184201,tt0000003,tt0000...",1,0,0,6,6.17
3,nm0721526,Émile Reynaud,tt0000004,Egy jó pohár sör (1892) - IMDb,1,0,None,6.6,90,1892-10-28,...,Émile Reynaud,1844,1918,director,"tt0413219,tt2184231,tt2184201,tt0000003,tt0000...",1,0,0,6,6.17
4,nm0005690,William K.L. Dickson,tt0000005,A patkolókovács (1893) - IMDb,18,3,None,6.2,1499,1893-05-09,...,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt0219560,tt1496763,tt0308254,tt1428455",0,1,0,4,5.75


In [163]:
# Identify records after merge with NaN values in director_names column

dropNaNNames = explodedAndMergeDF[explodedAndMergeDF.director_names.isna()]
dropNaNNames.shape

(12, 130)

In [164]:
# Drop the values without director_names
print("Dataframe Shape Before dropping NaN : ",str(explodedAndMergeDF.shape))
explodedAndMergeDF.drop(dropNaNNames.index, inplace= True)
print("Dataframe Shape After dropping NaN: ",str(explodedAndMergeDF.shape))

Dataframe Shape Before dropping NaN :  (798873, 130)
Dataframe Shape After dropping NaN:  (798861, 130)


In [165]:
explodedAndMergeDF.loc[explodedAndMergeDF['director_names'].str.contains('None')]

,director_ids,director_names,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,...,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,director_profession_primary,director_profession_secondary,director_profession_tertiary,director_known_titles_count,director_known_titles_average_rating_value


In [166]:
stage2ExplodeAndMergeDF = explodedAndMergeDF.sort_values(by ='movie_id')

In [167]:
print("stage2ExplodeAndMergeDF Dataframe Shape : ",str(stage2ExplodeAndMergeDF.shape))
stage2ExplodeAndMergeDF.head(2)

stage2ExplodeAndMergeDF Dataframe Shape :  (798861, 130)


,director_ids,director_names,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,...,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,director_profession_primary,director_profession_secondary,director_profession_tertiary,director_known_titles_count,director_known_titles_average_rating_value
0,nm0005690,William K.L. Dickson,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,...,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt0219560,tt1496763,tt0308254,tt1428455",0,1,0,4,5.75
1,nm0721526,Émile Reynaud,tt0000002,A bohóc és kutyái (1892) - IMDb,0,0,None,6.5,148,1892-10-28,...,Émile Reynaud,1844,1918,director,"tt0413219,tt2184231,tt2184201,tt0000003,tt0000...",1,0,0,6,6.17


In [168]:
# Search for a value in a column

stage2ExplodeAndMergeDF.loc[stage2ExplodeAndMergeDF['movie_id'] == 'tt0067806']

,director_ids,director_names,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,...,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,director_profession_primary,director_profession_secondary,director_profession_tertiary,director_known_titles_count,director_known_titles_average_rating_value
54086,nm0199254,Lucio Dandolo,tt0067806,Il suo nome era Pot (1971) - IMDb,1,0,None,3.7,23,1971-11-26,...,Lucio Dandolo,\N,\N,"director,writer","tt0204580,tt0067806,tt0073593",1,0,0,2,4.45
54087,nm0275717,Demofilo Fidani,tt0067806,Il suo nome era Pot (1971) - IMDb,1,0,None,3.7,23,1971-11-26,...,Demofilo Fidani,1914,1994,"writer,director,production_designer","tt0062311,tt0068162,tt0216546,tt0145020",0,1,0,4,4.47


In [169]:
stage2ExplodeAndMergeDF.nunique().head(15)

director_ids           174919
director_names         170704
movie_id               617969
title                  615421
review_count_user         999
review_count_critic       538
metascore                 101
rating_value               91
rating_count            14793
date_published          33832
duration                  595
title_year                132
release_date           187987
runtime                   561
directed_by            151380
dtype: int64

In [170]:
stage2ExplodeAndMergeDF.shape

(798861, 130)

In [171]:
# Clean column names 
modified_columns = stage2ExplodeAndMergeDF.columns.values
modified_columns = [col.replace(' ', '_') for col in modified_columns]
modified_columns = [col.replace('-', '_') for col in modified_columns]

stage2ExplodeAndMergeDF.columns = modified_columns

In [172]:
# save_to_file(stage2ExplodeAndMergeDF,folder_path)

In [173]:
modified_columns[:5]

['director_ids', 'director_names', 'movie_id', 'title', 'review_count_user']